# Парсинг данных с сайта РБК

## Импорт необходимых библиотек

In [1]:
import requests
import warnings
from bs4 import BeautifulSoup as bs
from bs4 import NavigableString, Tag
import pandas as pd
from tqdm.notebook import tqdm
from datetime import datetime

## Настройки среды

In [2]:
warnings.filterwarnings("ignore")
tqdm.pandas()
# Сброс ограничений на количество выводимых рядов
pd.set_option('display.max_rows', 20)
# Сброс ограничений на число столбцов
pd.set_option('display.max_columns', None)
# Сброс ограничений на количество символов в записи
pd.set_option('display.max_colwidth', None)

## Загрузка датасетов

In [3]:
train_data = pd.read_csv('train.csv', index_col='document_id')

In [4]:
train_data.head(3)

,title,publish_date,session,authors,ctr,category,tags,views,depth,full_reads_percent
document_id,,,,,,,,,,
624ac09c9a7947db3d80c98eIDE7mtH4RBqGn-8MXfGffQ,Европейский банк развития приостановил доступ Москвы и Минска к финансам,2022-04-04 10:29:44,IDE7mtH4RBqGn-8MXfGffQ,[],1.580,5409f11ce063da9c8b588a18,"['55928d339a794751dc8303d6', '542d1e28cbb20f862ac2c1f6', '5409f140e063daa0f408ad1d', '55797e339a794789759c64a1']",20460,1.134,35.850
620f6b899a7947701cf489e1KtVJsteHStO5oditt3Uvzw,Кремль назвал регулярным процессом учебные запуски баллистических ракет,2022-02-18 10:00:39,KtVJsteHStO5oditt3Uvzw,"['54244e01cbb20f03076b236d','5878a2ec9a7947e531faf52c']",1.853,5409f11ce063da9c8b588a12,"['549d25df9a794775979561d2', '58abcf539a7947f18bc079eb', '5409f196e063daa0f408ae67']",19038,1.142,38.355
620730cf9a7947ab96a44e27hk7puWJwSziw0m3sfTkKWA,"Госсекретарь Швеции заявила о нежелании вступать в НАТО из-за Украины\n \n \n\n \n\n Политика, 07:24",2022-02-12 04:24:02,hk7puWJwSziw0m3sfTkKWA,[],0.000,5409f11ce063da9c8b588a12,"['5430f451cbb20f73931ecd05', '5409f15de063daa0f408ada8', '5409f196e063daa0f408ae65']",51151,1.185,36.424


In [5]:
test_data = pd.read_csv('test.csv', index_col='document_id')

In [6]:
test_data.head(3)

,title,publish_date,session,authors,ctr,category,tags
document_id,,,,,,,
61f9569a9a794794245a82abJ0AvX96vTAaQCiWVbzoMdw,"«Крайне провокационная тема»: как Россия и США поспорили на Совбезе ООН\n \n \n\n \n\n Политика, 20:02",2022-02-01 17:02:44,J0AvX96vTAaQCiWVbzoMdw,[],0.000,5409f11ce063da9c8b588a12,"['58ca77679a794700b1e37f0a', '5409f1b7e063daa0f408aec4', '5409f15ee063daa0f408adb0', '5409f15de063daa0f408ada8', '5883d50f9a7947eac61a7dc5']"
628c22b89a79470e553f594bQS5CqzXYRnmDdR2LaSreEw,Власти Херсонской области пообещали сделать русский язык государственным,2022-05-24 00:50:55,QS5CqzXYRnmDdR2LaSreEw,[],1.598,5409f11ce063da9c8b588a12,"['5433603acbb20f6e5def0cc5', '5409f420e063daa0f408b5a5', '545f38dacbb20fe3c1b1fcce', '621a3d0c9a794728d449ae5e']"
627cb3249a7947ebdd752865XVsoyrUOT8OJJg2_finJhw,Премьер и президент Финляндии призвали срочно подать заявку в НАТО,2022-05-12 07:11:51,XVsoyrUOT8OJJg2_finJhw,[],2.006,5409f11ce063da9c8b588a12,"['5409f270e063daa0f408b0a7', '5409f196e063daa0f408ae65', '5430f451cbb20f73931ecd05']"


## Пердобработка датасетов

In [7]:
df_train = train_data.copy()
df_test = test_data.copy()

### document_id

In [8]:
def document_id_transform(dataframe):
    df = dataframe.copy()
    df['url_id'] = df.index
    for index, row in tqdm(df.iterrows()):
        df.at[index, 'url_id'] = row['url_id'].replace(row['session'], '')
    return df

In [9]:
df_train = document_id_transform(df_train)
df_test = document_id_transform(df_test)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [11]:
df_train.head(3)

,title,publish_date,session,authors,ctr,category,tags,views,depth,full_reads_percent,url_id
document_id,,,,,,,,,,,
624ac09c9a7947db3d80c98eIDE7mtH4RBqGn-8MXfGffQ,Европейский банк развития приостановил доступ Москвы и Минска к финансам,2022-04-04 10:29:44,IDE7mtH4RBqGn-8MXfGffQ,[],1.580,5409f11ce063da9c8b588a18,"['55928d339a794751dc8303d6', '542d1e28cbb20f862ac2c1f6', '5409f140e063daa0f408ad1d', '55797e339a794789759c64a1']",20460,1.134,35.850,624ac09c9a7947db3d80c98e
620f6b899a7947701cf489e1KtVJsteHStO5oditt3Uvzw,Кремль назвал регулярным процессом учебные запуски баллистических ракет,2022-02-18 10:00:39,KtVJsteHStO5oditt3Uvzw,"['54244e01cbb20f03076b236d','5878a2ec9a7947e531faf52c']",1.853,5409f11ce063da9c8b588a12,"['549d25df9a794775979561d2', '58abcf539a7947f18bc079eb', '5409f196e063daa0f408ae67']",19038,1.142,38.355,620f6b899a7947701cf489e1
620730cf9a7947ab96a44e27hk7puWJwSziw0m3sfTkKWA,"Госсекретарь Швеции заявила о нежелании вступать в НАТО из-за Украины\n \n \n\n \n\n Политика, 07:24",2022-02-12 04:24:02,hk7puWJwSziw0m3sfTkKWA,[],0.000,5409f11ce063da9c8b588a12,"['5430f451cbb20f73931ecd05', '5409f15de063daa0f408ada8', '5409f196e063daa0f408ae65']",51151,1.185,36.424,620730cf9a7947ab96a44e27


### title

In [12]:
def title_transform(dataframe):
    df = dataframe.copy()
    for index, row in tqdm(df.iterrows()):
        title = list(row['title'])
        for i in range(len(title)):
            if title[i] == '\n':
                temp = ''.join(title[:i])
                df.at[index, 'title'] = temp
                break
    return df

In [13]:
df_train = title_transform(df_train)
df_test = title_transform(df_test)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [14]:
df_train.head(3)

,title,publish_date,session,authors,ctr,category,tags,views,depth,full_reads_percent,url_id
document_id,,,,,,,,,,,
624ac09c9a7947db3d80c98eIDE7mtH4RBqGn-8MXfGffQ,Европейский банк развития приостановил доступ Москвы и Минска к финансам,2022-04-04 10:29:44,IDE7mtH4RBqGn-8MXfGffQ,[],1.580,5409f11ce063da9c8b588a18,"['55928d339a794751dc8303d6', '542d1e28cbb20f862ac2c1f6', '5409f140e063daa0f408ad1d', '55797e339a794789759c64a1']",20460,1.134,35.850,624ac09c9a7947db3d80c98e
620f6b899a7947701cf489e1KtVJsteHStO5oditt3Uvzw,Кремль назвал регулярным процессом учебные запуски баллистических ракет,2022-02-18 10:00:39,KtVJsteHStO5oditt3Uvzw,"['54244e01cbb20f03076b236d','5878a2ec9a7947e531faf52c']",1.853,5409f11ce063da9c8b588a12,"['549d25df9a794775979561d2', '58abcf539a7947f18bc079eb', '5409f196e063daa0f408ae67']",19038,1.142,38.355,620f6b899a7947701cf489e1
620730cf9a7947ab96a44e27hk7puWJwSziw0m3sfTkKWA,Госсекретарь Швеции заявила о нежелании вступать в НАТО из-за Украины,2022-02-12 04:24:02,hk7puWJwSziw0m3sfTkKWA,[],0.000,5409f11ce063da9c8b588a12,"['5430f451cbb20f73931ecd05', '5409f15de063daa0f408ada8', '5409f196e063daa0f408ae65']",51151,1.185,36.424,620730cf9a7947ab96a44e27


### publish_date

In [15]:
def publish_date_transform(dataframe):
    df = dataframe.copy()
    df['publish_date'] = df['publish_date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
    return df

In [16]:
df_train = publish_date_transform(df_train)
df_test = publish_date_transform(df_test)

In [17]:
df_train.head(3)

,title,publish_date,session,authors,ctr,category,tags,views,depth,full_reads_percent,url_id
document_id,,,,,,,,,,,
624ac09c9a7947db3d80c98eIDE7mtH4RBqGn-8MXfGffQ,Европейский банк развития приостановил доступ Москвы и Минска к финансам,2022-04-04 10:29:44,IDE7mtH4RBqGn-8MXfGffQ,[],1.580,5409f11ce063da9c8b588a18,"['55928d339a794751dc8303d6', '542d1e28cbb20f862ac2c1f6', '5409f140e063daa0f408ad1d', '55797e339a794789759c64a1']",20460,1.134,35.850,624ac09c9a7947db3d80c98e
620f6b899a7947701cf489e1KtVJsteHStO5oditt3Uvzw,Кремль назвал регулярным процессом учебные запуски баллистических ракет,2022-02-18 10:00:39,KtVJsteHStO5oditt3Uvzw,"['54244e01cbb20f03076b236d','5878a2ec9a7947e531faf52c']",1.853,5409f11ce063da9c8b588a12,"['549d25df9a794775979561d2', '58abcf539a7947f18bc079eb', '5409f196e063daa0f408ae67']",19038,1.142,38.355,620f6b899a7947701cf489e1
620730cf9a7947ab96a44e27hk7puWJwSziw0m3sfTkKWA,Госсекретарь Швеции заявила о нежелании вступать в НАТО из-за Украины,2022-02-12 04:24:02,hk7puWJwSziw0m3sfTkKWA,[],0.000,5409f11ce063da9c8b588a12,"['5430f451cbb20f73931ecd05', '5409f15de063daa0f408ada8', '5409f196e063daa0f408ae65']",51151,1.185,36.424,620730cf9a7947ab96a44e27


### authors

In [18]:
def cleaner_authors(raw):
    if raw != '[]':
        raw = raw.replace('[', '')
        raw = raw.replace(']', '')
        raw = raw.replace('\'', '')
        raw = raw.replace(' ', '')
    else:
        raw = 'no_authors'
    return raw

In [19]:
def authors_transfrom(dataframe):
    df = dataframe.copy()
    df['authors'] = df['authors'].apply(lambda x: cleaner_authors(x))
    return df

In [20]:
df_train = authors_transfrom(df_train)
df_test = authors_transfrom(df_test)

In [21]:
df_train.head(3)

,title,publish_date,session,authors,ctr,category,tags,views,depth,full_reads_percent,url_id
document_id,,,,,,,,,,,
624ac09c9a7947db3d80c98eIDE7mtH4RBqGn-8MXfGffQ,Европейский банк развития приостановил доступ Москвы и Минска к финансам,2022-04-04 10:29:44,IDE7mtH4RBqGn-8MXfGffQ,no_authors,1.580,5409f11ce063da9c8b588a18,"['55928d339a794751dc8303d6', '542d1e28cbb20f862ac2c1f6', '5409f140e063daa0f408ad1d', '55797e339a794789759c64a1']",20460,1.134,35.850,624ac09c9a7947db3d80c98e
620f6b899a7947701cf489e1KtVJsteHStO5oditt3Uvzw,Кремль назвал регулярным процессом учебные запуски баллистических ракет,2022-02-18 10:00:39,KtVJsteHStO5oditt3Uvzw,"54244e01cbb20f03076b236d,5878a2ec9a7947e531faf52c",1.853,5409f11ce063da9c8b588a12,"['549d25df9a794775979561d2', '58abcf539a7947f18bc079eb', '5409f196e063daa0f408ae67']",19038,1.142,38.355,620f6b899a7947701cf489e1
620730cf9a7947ab96a44e27hk7puWJwSziw0m3sfTkKWA,Госсекретарь Швеции заявила о нежелании вступать в НАТО из-за Украины,2022-02-12 04:24:02,hk7puWJwSziw0m3sfTkKWA,no_authors,0.000,5409f11ce063da9c8b588a12,"['5430f451cbb20f73931ecd05', '5409f15de063daa0f408ada8', '5409f196e063daa0f408ae65']",51151,1.185,36.424,620730cf9a7947ab96a44e27


### category

In [22]:
categories_name = {
    '5409f11ce063da9c8b588a12': 'politics',
    '5433e5decbb20f277b20eca9': 'society',
    '540d5eafcbb20f2524fc0509': 'business',
    '5409f11ce063da9c8b588a13': 'economics',
    '540d5ecacbb20f2524fc050a': 'technology_and_media',
    '5409f11ce063da9c8b588a18': 'finances',
    '552e430f9a79475dd957f8b3': 'money',
    '5e54e2089a7947f63a801742': 'auto',
    '5e54e22a9a7947f560081ea2': 'realty'
}

In [23]:
def category_transform(dataframe):
    df = dataframe.copy()
    df['category'] = df['category'].apply(lambda x: categories_name[x])
    return df

In [24]:
df_train = category_transform(df_train)
df_test = category_transform(df_test)

In [25]:
df_train.head(3)

,title,publish_date,session,authors,ctr,category,tags,views,depth,full_reads_percent,url_id
document_id,,,,,,,,,,,
624ac09c9a7947db3d80c98eIDE7mtH4RBqGn-8MXfGffQ,Европейский банк развития приостановил доступ Москвы и Минска к финансам,2022-04-04 10:29:44,IDE7mtH4RBqGn-8MXfGffQ,no_authors,1.580,finances,"['55928d339a794751dc8303d6', '542d1e28cbb20f862ac2c1f6', '5409f140e063daa0f408ad1d', '55797e339a794789759c64a1']",20460,1.134,35.850,624ac09c9a7947db3d80c98e
620f6b899a7947701cf489e1KtVJsteHStO5oditt3Uvzw,Кремль назвал регулярным процессом учебные запуски баллистических ракет,2022-02-18 10:00:39,KtVJsteHStO5oditt3Uvzw,"54244e01cbb20f03076b236d,5878a2ec9a7947e531faf52c",1.853,politics,"['549d25df9a794775979561d2', '58abcf539a7947f18bc079eb', '5409f196e063daa0f408ae67']",19038,1.142,38.355,620f6b899a7947701cf489e1
620730cf9a7947ab96a44e27hk7puWJwSziw0m3sfTkKWA,Госсекретарь Швеции заявила о нежелании вступать в НАТО из-за Украины,2022-02-12 04:24:02,hk7puWJwSziw0m3sfTkKWA,no_authors,0.000,politics,"['5430f451cbb20f73931ecd05', '5409f15de063daa0f408ada8', '5409f196e063daa0f408ae65']",51151,1.185,36.424,620730cf9a7947ab96a44e27


### tags

In [26]:
def cleaner_tags(raw):
    if raw != '[]':
        raw = raw.replace('[', '')
        raw = raw.replace(']', '')
        raw = raw.replace('\'', '')
        raw = raw.replace(' ', '')
    else:
        raw = 'no_tags'
    return raw

In [27]:
def tags_transfrom(dataframe):
    df = dataframe.copy()
    df['tags'] = df['tags'].apply(lambda x: cleaner_tags(x))
    return df

In [28]:
df_train = tags_transfrom(df_train)
df_test = tags_transfrom(df_test)

In [29]:
df_train.head(3)

,title,publish_date,session,authors,ctr,category,tags,views,depth,full_reads_percent,url_id
document_id,,,,,,,,,,,
624ac09c9a7947db3d80c98eIDE7mtH4RBqGn-8MXfGffQ,Европейский банк развития приостановил доступ Москвы и Минска к финансам,2022-04-04 10:29:44,IDE7mtH4RBqGn-8MXfGffQ,no_authors,1.580,finances,"55928d339a794751dc8303d6,542d1e28cbb20f862ac2c1f6,5409f140e063daa0f408ad1d,55797e339a794789759c64a1",20460,1.134,35.850,624ac09c9a7947db3d80c98e
620f6b899a7947701cf489e1KtVJsteHStO5oditt3Uvzw,Кремль назвал регулярным процессом учебные запуски баллистических ракет,2022-02-18 10:00:39,KtVJsteHStO5oditt3Uvzw,"54244e01cbb20f03076b236d,5878a2ec9a7947e531faf52c",1.853,politics,"549d25df9a794775979561d2,58abcf539a7947f18bc079eb,5409f196e063daa0f408ae67",19038,1.142,38.355,620f6b899a7947701cf489e1
620730cf9a7947ab96a44e27hk7puWJwSziw0m3sfTkKWA,Госсекретарь Швеции заявила о нежелании вступать в НАТО из-за Украины,2022-02-12 04:24:02,hk7puWJwSziw0m3sfTkKWA,no_authors,0.000,politics,"5430f451cbb20f73931ecd05,5409f15de063daa0f408ada8,5409f196e063daa0f408ae65",51151,1.185,36.424,620730cf9a7947ab96a44e27


## Парсинг

### URL

In [31]:
def make_url(dataframe):
    df = dataframe.copy()
    df['url'] = df['url_id'].progress_apply(lambda x: f"https://www.rbc.ru/rbcfreenews/{x}")
    #df['status_code'] = df['url'].progress_apply(lambda x: requests.get(x).status_code)
    return df

In [32]:
df_train = make_url(df_train)
df_test = make_url(df_test)

  0%|          | 0/7000 [00:00<?, ?it/s]

  0%|          | 0/3000 [00:00<?, ?it/s]

In [33]:
df_train.head(3)

,title,publish_date,session,authors,ctr,category,tags,views,depth,full_reads_percent,url_id,url
document_id,,,,,,,,,,,,
624ac09c9a7947db3d80c98eIDE7mtH4RBqGn-8MXfGffQ,Европейский банк развития приостановил доступ Москвы и Минска к финансам,2022-04-04 10:29:44,IDE7mtH4RBqGn-8MXfGffQ,no_authors,1.580,finances,"55928d339a794751dc8303d6,542d1e28cbb20f862ac2c1f6,5409f140e063daa0f408ad1d,55797e339a794789759c64a1",20460,1.134,35.850,624ac09c9a7947db3d80c98e,https://www.rbc.ru/rbcfreenews/624ac09c9a7947db3d80c98e
620f6b899a7947701cf489e1KtVJsteHStO5oditt3Uvzw,Кремль назвал регулярным процессом учебные запуски баллистических ракет,2022-02-18 10:00:39,KtVJsteHStO5oditt3Uvzw,"54244e01cbb20f03076b236d,5878a2ec9a7947e531faf52c",1.853,politics,"549d25df9a794775979561d2,58abcf539a7947f18bc079eb,5409f196e063daa0f408ae67",19038,1.142,38.355,620f6b899a7947701cf489e1,https://www.rbc.ru/rbcfreenews/620f6b899a7947701cf489e1
620730cf9a7947ab96a44e27hk7puWJwSziw0m3sfTkKWA,Госсекретарь Швеции заявила о нежелании вступать в НАТО из-за Украины,2022-02-12 04:24:02,hk7puWJwSziw0m3sfTkKWA,no_authors,0.000,politics,"5430f451cbb20f73931ecd05,5409f15de063daa0f408ada8,5409f196e063daa0f408ae65",51151,1.185,36.424,620730cf9a7947ab96a44e27,https://www.rbc.ru/rbcfreenews/620730cf9a7947ab96a44e27


In [35]:
def article_parsing(url):
    # получаем html код статьи
    r = requests.get(url)
    #status_code = r.status_code
    soup = bs(r.text, 'lxml')
    article = soup.find('div', class_='l-col-main')
    article_body = article.find('div', class_='article__text article__text_free')
    
    # парсинг div в статье
    article_div = article_body.find_all('div')
    article_div = len(article_div)
    #print(article_div)
    
    # парсинг обзора статьи
    article_overview = article_body.find('div', class_='article__text__overview')
    if article_overview:
        article_overview = article.find('div', class_='article__text__overview').find('span').text
    else:
        article_overview = 'no_overview'
    #print(article_overview)
    
    # парсинг главного изображения статьи
    article_main_image = article.find('div', class_='article__main-image__wrap')
    if article_main_image:
        article_main_image = 1
    else:
        article_main_image = 0
    #print(article_main_image)
    
    # парсинг встроенных элементов статьи
    article_inline_items = article_body.find_all('div', class_='article__inline-item')
    article_inline_items = len(article_inline_items)
    #print(article_inline_items)
    
    # парсинг ссылок
    unwanted_urls = [['banner__container__link'], ['article__inline-item__link'], ['article__inline-item__category'],
                     ['article__inline-item__image-block', 'article__inline-item__link'], ['pro-anons__title'],
                     ['pro-anons__item__link', 'js-pro-anons-link', 'js-yandex-counter'], ['pro-anons__item__bottom__type'],]
    article_urls = []
    for url in article_body.find_all('a'):
        attrs = url.attrs
        if 'class' in attrs.keys():
            if url.attrs['class'] not in unwanted_urls:
                #print(url.attrs, url.text)
                article_urls.append(url)
        else:
            #print(url.attrs, url.text)
            article_urls.append(url)
    article_urls = len(article_urls)
    #print(article_urls)
    
    # парсинг рбк pro
    article_rbk_pro = article_body.find('div', class_='pro-anons__title-wrap js-pro-anons-scroll-arrow-wrap')
    if article_rbk_pro:
        article_rbk_pro = 1
    else:
        article_rbk_pro = 0
    #print(article_rbk_pro)
    
    # парсинг рекомендованого видео
    article_recommend_video = article_body.find('div', class_='js-video-autoplay-recommend js-yandex-counter')
    if article_recommend_video:
        article_recommend_video = 1
    else:
        article_recommend_video = 0
    #print(article_recommend_video)
    
    # парсинг видео в статье
    article_video = article_body.find_all('div', class_='article__inline-video js-insert-video-container')
    article_video.extend(article_body.find_all('iframe'))
    article_video = len(article_video)
    #print(article_video)
    
    # парсинг галереи в статье
    article_gallery = article_body.find('div', class_='gallery_vertical')
    if article_gallery:
        article_gallery = 1
    else:
        article_gallery = 0
    #print(article_gallery)
    
    # парсинг изображений в статье
    article_images = article_body.find_all('div', class_='article__picture')
    if article_gallery == 1:
        article_gallery_images = article_body.find_all('div', class_='gallery_vertical__item')
        article_images = len(article_images) + len(article_gallery_images)
    else:
        article_images = len(article_images)
    #print(article_images)
    
    # парсинг тегов
    article_tags = article.find('div', class_='article__tags__container')
    if article_tags:
        article_tags = article_tags.find_all('a')
        article_tags = ','.join([tag.text for tag in article_tags])
    else:
        article_tags = 'no_tags'
    #print(article_tags)
                                          
    # парсинг текста статьи
    article_text = article.find('div', class_='article__text article__text_free')
    divs_remove = ['article__text__overview', 'article__main-image', 'fox-tail', 'article__inline-item', 'pro-anons__wrapper', 'gallery_vertical__meta']
    for d in divs_remove:
        for el in article_text.find_all('div', class_=d):
            if el:
                el.decompose()
    span_remove = ['g-hidden']
    for s in span_remove:
        for el in article_text.find_all('span', class_=s):
            if el:
                el.decompose()
    article_text_test = article_text.text.replace('\n', '').replace(' ', '').replace('\r', '').replace('\xa0', '')
    if not article_text_test:
        article_text = 'no_text'
    else:
        article_text = article_text.text.replace('\n', ' ')
    #print(article_text)

    return (article_div, article_overview, article_main_image, article_inline_items, article_urls, article_rbk_pro,
            article_recommend_video, article_video, article_gallery, article_images, article_tags, article_text)
    

In [41]:
def parsing(dataframe):
    df = dataframe.copy()
    df['div_count'] = 0
    df['overview'] = 0
    df['main_image'] = 0
    df['inline_items'] = 0
    df['url_count'] = 0
    df['rbk_pro'] = 0
    df['recommend_video'] = 0
    df['video_count'] = 0
    df['gallery'] = 0
    df['image_count'] = 0
    df['tags_word'] = 0
    df['text'] = 0
    for index, row in tqdm(df.iterrows()):
        while True:
            try:
                dc, ov, mi, ii, uc, rp, rv, vc, g, ic, tw, tx = article_parsing(row['url'])
                df.at[index, 'div_count'] = dc
                df.at[index, 'overview'] = ov
                df.at[index, 'main_image'] = mi
                df.at[index, 'inline_items'] = ii
                df.at[index, 'url_count'] = uc
                df.at[index, 'rbk_pro'] = rp
                df.at[index, 'recommend_video'] = rv
                df.at[index, 'video_count'] = vc
                df.at[index, 'gallery'] = g
                df.at[index, 'image_count'] = ic
                df.at[index, 'tags_word'] = tw
                df.at[index, 'text'] = tx
            except:
                continue
            break
    return df

In [42]:
df_train = parsing(df_train)

0it [00:00, ?it/s]

In [43]:
df_train.head(3)

,title,publish_date,session,authors,ctr,category,tags,views,depth,full_reads_percent,url_id,url,div_count,overview,main_image,inline_items,url_count,rbk_pro,recommend_video,video_count,gallery,image_count,tags_word,text
document_id,,,,,,,,,,,,,,,,,,,,,,,,
624ac09c9a7947db3d80c98eIDE7mtH4RBqGn-8MXfGffQ,Европейский банк развития приостановил доступ Москвы и Минска к финансам,2022-04-04 10:29:44,IDE7mtH4RBqGn-8MXfGffQ,no_authors,1.580,finances,"55928d339a794751dc8303d6,542d1e28cbb20f862ac2c1f6,5409f140e063daa0f408ad1d,55797e339a794789759c64a1",20460,1.134,35.850,624ac09c9a7947db3d80c98e,https://www.rbc.ru/rbcfreenews/624ac09c9a7947db3d80c98e,69,В заявлении банка говорится: «Ни в одной из этих стран не может быть нового финансирования проектов или деятельности по техническому сотрудничеству»,1,1,2,1,1,0,0,0,"Европейский банк реконструкции и развития,Белоруссия,Россия,антироссийские санкции","Совет управляющих Европейского банка реконструкции и развития (ЕБРР) проголосовал за приостановку доступа России и Белоруссии к своим финансам и экспертизе, сообщила пресс-служба финансовой организации. «Доступ Российской Федерации и Белоруссии к ресурсам банка будет официально и немедленно приостановлен после вступления в силу соответствующего решения управляющих», — указано в сообщении. В банке пояснили, что ограничения также запрещают участие банка в любых проектах России или Белоруссии, в том числе в рамках технического сотрудничества. ЕБРР также объявил, что «оставляет за собой право на приостановку или полное прекращение выборки средств по существующим проектам». После начала спецоперации России на Украине и введения против нее санкций ЕБРР объявил о закрытии своих офисов в Москве и Минске. Банк также согласовал первоначальный пакет помощи в размере €2 млрд для Украины и других стран. В 2017 году ЕБРР анонсировал закрытие всех российских региональных отделений, кроме московского и петербургского. Банк ЕБРР был создан по инициативе президента Франции Франсуа Миттерана для перехода к рыночной экономике и развития частной и предпринимательской инициативы в странах Центральной и Восточной Европы. Финансовые операции банк начал проводить с 1991 года. В настоящее время акционерами ЕБРР является 71 страна, а также Европейский союз и Европейский инвестиционный банк. Штаб-квартира организации находится в Лондоне. ЕБРР вел операции в России с 1992 года, в 2014-м совет директоров банка принял решение о постепенном сворачивании проектов в России. В 2018 году объем его вложений в стране составлял $3,5 млрд."
620f6b899a7947701cf489e1KtVJsteHStO5oditt3Uvzw,Кремль назвал регулярным процессом учебные запуски баллистических ракет,2022-02-18 10:00:39,KtVJsteHStO5oditt3Uvzw,"54244e01cbb20f03076b236d,5878a2ec9a7947e531faf52c",1.853,politics,"549d25df9a794775979561d2,58abcf539a7947f18bc079eb,5409f196e063daa0f408ae67",19038,1.142,38.355,620f6b899a7947701cf489e1,https://www.rbc.ru/rbcfreenews/620f6b899a7947701cf489e1,11,"Все страны, которым положено знать об учениях, уже уведомлены о них, сообщил Песков",1,1,0,0,0,0,0,0,"РВСН,Владимир Путин,учения","Учения стратегических сил с запусками баллистических ракет, которые пройдут завтра под командованием Владимира Путина, — это «регулярный тренировочный процесс», о котором были заранее уведомлены все, кто должен об этом знать. Об этом заявил журналистам пресс-секретарь президента Дмитрий Песков, передает корреспондент РБК. «Путин будет находиться в ситуационном центре. Подобные пуски невозможны без главы государства. Вы знаете про знаменитый «черный чемоданчик», «красную кнопку» и тому подобное», — рассказал Песков. Он подчеркнул, что никакой озабоченности плановые учения вызывать не должны. «Вы видите, что сейчас идет целый ряд учений, и, опять же, это те действия, которые абсолютно прозрачны, абсолютно понятны для специалистов из других стран. Поэтому они не должны вызывать ни у кого обеспокоенности», — сказал Песков. Он рассказал, что таким учениям предшествует целая серия уведомлений различным странам, весь проце

In [44]:
df_test = parsing(df_test)

0it [00:00, ?it/s]

In [45]:
df_test.head(3)

title  \
document_id                                                                                                                
61f9569a9a794794245a82abJ0AvX96vTAaQCiWVbzoMdw   «Крайне провокационная тема»: как Россия и США поспорили на Совбезе ООН   
628c22b89a79470e553f594bQS5CqzXYRnmDdR2LaSreEw  Власти Херсонской области пообещали сделать русский язык государственным   
627cb3249a7947ebdd752865XVsoyrUOT8OJJg2_finJhw        Премьер и президент Финляндии призвали срочно подать заявку в НАТО   

                                                      publish_date  \
document_id                                                          
61f9569a9a794794245a82abJ0AvX96vTAaQCiWVbzoMdw 2022-02-01 17:02:44   
628c22b89a79470e553f594bQS5CqzXYRnmDdR2LaSreEw 2022-05-24 00:50:55   
627cb3249a7947ebdd752865XVsoyrUOT8OJJg2_finJhw 2022-05-12 07:11:51   

                                                               session  \
document_id                                                              
61f9569a9a794794245a82abJ0AvX96vTAaQCiWVbzoMdw  J0AvX96vTAaQCiWVbzoMdw   
628c22b89a79470e553f594bQS5CqzXYRnmDdR2LaSreEw  QS5CqzXYRnmDdR2LaSreEw   
627cb3249a7947ebdd752865XVsoyrUOT8OJJg2_finJhw  XVsoyrUOT8OJJg2_finJhw   

                                                   authors    ctr  category  \
document_id                                                                   
61f9569a9a794794245a82abJ0AvX96vTAaQCiWVbzoMdw  no_authors  0.000  politics   
628c22b89a79470e553f594bQS5CqzXYRnmDdR2LaSreEw  no_authors  1.598  politics   
627cb3249a7947ebdd752865XVsoyrUOT8OJJg2_finJhw  no_authors  2.006  politics   

                                                                                                                                                                        tags  \
document_id                                                                                                                                                                    
61f9569a9a794794245a82abJ0AvX96vTAaQCiWVbzoMdw  58ca77679a794700b1e37f0a,5409f1b7e063daa0f408aec4,5409f15ee063daa0f408adb0,5409f15de063daa0f408ada8,5883d50f9a7947eac61a7dc5   
628c22b89a79470e553f594bQS5CqzXYRnmDdR2LaSreEw                           5433603acbb20f6e5def0cc5,5409f420e063daa0f408b5a5,545f38dacbb20fe3c1b1fcce,621a3d0c9a794728d449ae5e   
627cb3249a7947ebdd752865XVsoyrUOT8OJJg2_finJhw                                                    5409f270e063daa0f408b0a7,5409f196e063daa0f408ae65,5430f451cbb20f73931ecd05   

                                                                  url_id  \
document_id                                                                
61f9569a9a794794245a82abJ0AvX96vTAaQCiWVbzoMdw  61f9569a9a794794245a82ab   
628c22b89a79470e553f594bQS5CqzXYRnmDdR2LaSreEw  628c22b89a79470e553f594b   
627cb3249a7947ebdd752865XVsoyrUOT8OJJg2_finJhw  627cb3249a7947ebdd752865   

                                                                                                    url  \
document_id                                                                                               
61f9569a9a794794245a82abJ0AvX96vTAaQCiWVbzoMdw  https://www.rbc.ru/rbcfreenews/61f9569a9a794794245a82ab   
628c22b89a79470e553f594bQS5CqzXYRnmDdR2LaSreEw  https://www.rbc.ru/rbcfreenews/628c22b89a79470e553f594b   
627cb3249a7947ebdd752865XVsoyrUOT8OJJg2_finJhw  https://www.rbc.ru/rbcfreenews/627cb3249a7947ebdd752865   

                                                div_count  \
document_id                                                 
61f9569a9a794794245a82abJ0AvX96vTAaQCiWVbzoMdw          1   
628c22b89a79470e553f594bQS5CqzXYRnmDdR2LaSreEw         73   
627cb3249a7947ebdd752865XVsoyrUOT8OJJg2_finJhw         72   

                                                                                                                                                                                                                                                             overview  \
documen

In [46]:
df_train.to_csv('train_pr_v2.csv')

In [47]:
df_test.to_csv('test_pr_v2.csv')